In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv

In this case, we will try to lane roads

In [2]:
class Vehicle:
    def __init__(self, vehicle_id, position, speed, max_speed, braking_prob):
        self.vehicle_id = vehicle_id
        self.position = position
        self.speed = speed
        self.max_speed = max_speed
        self.braking_prob = braking_prob

    def accelerate(self):
        if self.speed < self.max_speed:
            self.speed += 1

    def decelerate(self, gap_distance):
        if self.speed > gap_distance:
            self.speed = gap_distance

    def randomize(self, braking_prob):
        if np.random.rand() < braking_prob:
            if self.speed > 0:
                self.speed -= 1

In [3]:
class Road_Configuration:
    def __init__(self, road_length, road_width, production_prob, max_speed):
        self.road_length = road_length
        self.road_width = road_width
        self.production_prob = production_prob
        self.max_speed = max_speed
        self.road_occupancy = np.zeros((self.road_length, self.road_width), dtype=int)
        self.vehicles = []
        self.vehicle_counter = 0  # To assign unique IDs to vehicles

    def update_occupancy(self):
        self.road_occupancy.fill(0)  # Reset occupancy grid
        for vehicle in self.vehicles:
            if vehicle.position < self.road_length:  # Only update occupancy if within road length
                self.road_occupancy[vehicle.position, 0] = 1  # Assuming single-lane for simplicity

    def produce_vehicles(self):
        if np.random.rand() < self.production_prob:
            if np.sum(self.road_occupancy[0, :]) == 0:  # Check the availability of the first cell
                new_vehicle = Vehicle(self.vehicle_counter, 0, np.random.randint(1, self.max_speed + 1), self.max_speed, 0.01)
                self.vehicle_counter += 1
                self.vehicles.append(new_vehicle)
                self.road_occupancy[new_vehicle.position, 0] = 1  # Mark the position as occupied

    def gap_distance(self, vehicle):
        current_position = vehicle.position
        for distance in range(1, self.road_length):
            next_position = current_position + distance
            if next_position >= self.road_length:
                break
            if self.road_occupancy[next_position, 0] == 1:
                return distance - 1
        return self.road_length - 1

    def update(self): #updates the road configuration based on nagel-schreckenberg rules
        self.update_occupancy()
        for vehicle in self.vehicles:
            distance_to_next = self.gap_distance(vehicle)
            vehicle.accelerate()
            vehicle.decelerate(distance_to_next)
            vehicle.randomize(vehicle.braking_prob)
            vehicle.position += vehicle.speed

        self.update_occupancy()

    def vehicle_density(self): #computes the density of the road for data processing
        return len(self.vehicles) / self.road_length